In [683]:
import pandas as pd
import numpy as np
import random
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from tqdm import tqdm
import gudhi as gd

In [684]:
# print("Loading the dataset...")
df = pd.read_csv(r"../data/preprocessed_data.csv")
# print(f"Original dataset loaded with {len(df)} records.")

# Log the intention to sample
# print("Attempting to sample 1000 records with replacement...")
try:
    df_sampled = df.sample(n=10000, replace=True)  # Allows sampling the same row more than once
    # print(f"Sampled dataset created with {len(df_sampled)} records.")
except ValueError as e:
    print(f"Error: Not enough records to sample with replacement: {e}")
    exit()

In [685]:
ga_type = input("Enter GA type (generic (1) or proposed (2)): ").strip()
# ga_type = "1"

if ga_type not in ['1', '2']:
    raise ValueError("Invalid GA type. Please enter '1' for generic or '2' for proposed.")


# GENERIC GENETIC ALGO


In [686]:
def split_timeslot(timeslot):
    parts = timeslot.split(' ')
    if len(parts) >= 2:
        return parts[0], ' '.join(parts[1:])
    return timeslot, ''  # Adjust this according to the expected format

df_sampled['day'], df_sampled['time'] = zip(*df_sampled['timeslot'].apply(split_timeslot))
df_sampled['start_time'], df_sampled['end_time'] = zip(*df_sampled['time'].apply(lambda x: x.split('-') if x else ('', '')))

In [687]:
class Schedule:
    # Class variable to track all occupied timeslots
    occupied_timeslots = set()

    def __init__(self, assignments, teacher_type, ga_type, max_hours=None, unavailable_days=None, location_to_index_map=None):
        self.assignments = assignments
        self.teacher_type = teacher_type
        self.ga_type = ga_type
        self.max_hours = max_hours
        self.unavailable_days = unavailable_days or []
        self.location_to_index_map = location_to_index_map
        self.preprocessed_data = []  # Initialize preprocessed_data
        self.preprocess_assignments()  # Preprocess assignments

    def update_occupied_timeslots(self):
        for a in self.assignments:
            Schedule.occupied_timeslots.add((a['faculty'], a['timeslot']))

    def is_timeslot_available(self, faculty, timeslot):
        return (faculty, timeslot) not in Schedule.occupied_timeslots

    def __str__(self):
        schedule_details = "Schedule:\n"
        for assignment in self.assignments:
            if isinstance(assignment, dict):
                schedule_details += f"Course: {str(assignment.get('course', ''))}, Faculty: {str(assignment.get('faculty', ''))}, Timeslot: {str(assignment.get('timeslot', ''))}, Classroom: {str(assignment.get('classroom', ''))}\n"
            return schedule_details

    def fitness(self):
        if self.ga_type == '1':
            return self.generic_fitness_evaluation()
        elif self.ga_type == '2':
            return self.tda_based_fitness_evaluation()
        else:
            print(f"Unexpected GA type encountered in fitness evaluation: {self.ga_type}")
            return 100  # Or handle as appropriate


    def generic_fitness_evaluation(self):
        # Initialize variables
        total_hours = 0
        faculty_course_count = {}
        penalty = 0

        for a in self.assignments:
            # print("== Inside (for a in self.assignments loop) ==")

            # print(f"a: {a}")

            duration = self.calculate_duration(a['start_time'], a['end_time'])
            # print(f"duration: {duration}")

            total_hours += duration
            # print(f"total_hours: {total_hours}")

            faculty_course_count[a['faculty']] = faculty_course_count.get(a['faculty'], 0) + 1
            # print(f"faculty_course_count: {faculty_course_count}")


        # print(f"1 total_hours: {total_hours}")
        # print(f"2 faculty_course_count: {faculty_course_count}")

        # Calculate penalties
        # Overloading Faculty
        if self.teacher_type == 'FT' and total_hours > self.max_hours:
            penalty += 10

        # Part-time Faculty
        if self.teacher_type == 'PT' and total_hours > 12:
            penalty += 7

        # Inequitable Course Distribution
        print(f"faculty_course_count: {faculty_course_count}")
        # faculty_course_count: {'FT6': 2, 'PT1': 3, 'PT4': 2, 'FT2': 1, 'FT3': 1, 'FT1': 1}

        # Check if distribution is inequitable
        if len(faculty_course_count) > 1:
            max_courses = max(faculty_course_count.values())
            min_courses = min(faculty_course_count.values())
            error_percentage = (max_courses - min_courses) / max_courses

            if error_percentage > 0.1:
                penalty += int(error_percentage * 10)  # Adjust the penalty based on the error percentage

        # Consulting Hours Overlap
        for i in range(len(self.assignments)):
            for j in range(i + 1, len(self.assignments)):
                if self.assignments[i]['faculty'] == self.assignments[j]['faculty'] and \
                        self.assignments[i]['timeslot'] == self.assignments[j]['timeslot']:
                    penalty += 5
                    break

        # Faculty Preferences Ignored
        for a in self.assignments:
            if a['day'] in self.unavailable_days:
                penalty += 4

        # Classroom Utilization Inefficiency - Underutilized or overutilized classrooms
        classroom_usage = {}
        for a in self.assignments:
            classroom_usage[a['classroom']] = classroom_usage.get(a['classroom'], 0) + self.calculate_duration(a['start_time'], a['end_time'])

        for classroom, usage in classroom_usage.items():
            if usage < 3:
                penalty += 3
            elif usage > 5:
                penalty += 3

        # Student Schedule Clashes
        for i in range(len(self.assignments)):
            for j in range(i + 1, len(self.assignments)):
                if self.assignments[i]['timeslot'] == self.assignments[j]['timeslot'] and \
                        self.assignments[i]['classroom'] == self.assignments[j]['classroom']:
                    penalty += 2
                    break

        # Compute fitness score and subtract penalties
        fitness_score = 1 / (1 + penalty)

        print(f"4 fitness_score (Must not stay on same values always): {fitness_score}")

        return fitness_score

    @staticmethod
    def calculate_duration(start, end):
        def parse_time(time_str):
            time_str = time_str.strip()
            if ' PM' in time_str or ' AM' in time_str:
                time_str = time_str[:time_str.rfind(' ')]
            for fmt in ('%I:%M %p', '%H:%M'):
                try:
                    return datetime.strptime(time_str, fmt)
                except ValueError:
                    continue
            raise ValueError(f"Time data '{time_str}' does not match expected format")

        start_time, end_time = parse_time(start), parse_time(end)
        if end_time < start_time:
            end_time += timedelta(days=1)

        return (end_time - start_time).total_seconds() / 3600.0 # In hours

    @staticmethod
    def convert_time_to_float(time_str):
        # Map each day to a number (e.g., Monday=0, Tuesday=1, etc.)
        day_to_num = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

        # Split the time string into day and time of day (if day is provided)
        parts = time_str.strip().split(maxsplit=1)
        day_part = parts[0] if len(parts) == 2 else "Monday"  # Default to Monday if day is not provided
        time_of_day = parts[-1]

        day_num = day_to_num.get(day_part, 0)  # Default to 0 if day is not found

        # Convert time to total hours
        time_format = "%H:%M"
        time_delta = datetime.strptime(time_of_day, time_format) - datetime.strptime("00:00", time_format)
        total_hours = 24 * day_num + time_delta.total_seconds() / 3600

        return total_hours

    @staticmethod
    def convert_location_to_index(location_str, location_to_index_map):
        # Assuming location_to_index_map is a pre-defined dictionary mapping locations to unique integers
        # return location_to_index_map.get(location_str, -1)  # Return -1 or some default value if location is not found
        print('Test')

    def preprocess_assignments(self):
        """
        Preprocess and store time and classroom index information for each assignment.
        """
        for assignment in self.assignments:
            if isinstance(assignment, dict):
                time_float = self.convert_time_to_float(assignment.get('start_time', ''))
                classroom_index = self.convert_location_to_index(assignment.get('classroom', ''), self.location_to_index_map)
                self.preprocessed_data.append((time_float, classroom_index))
                print(f"Processed assignment: {assignment}, Time Float: {time_float}, Classroom Index: {classroom_index}")  # Debugging print

    def to_point_cloud(self):
        """
        Generate a point cloud from preprocessed assignment data.
        """
        # Use preprocessed data
        return self.preprocessed_data

    def tda_based_fitness_evaluation(self):
        # # Initialize variables
        # total_hours = 0
        # faculty_course_count = {}
        # penalty = 0

        # # Faculty Preferences Ignored
        # for a in self.assignments:
        #     if isinstance(a, dict) and a['day'] in self.unavailable_days:
        #         penalty += 4

        # # print(f"1 total_hours: {total_hours}")
        # # print(f"2 faculty_course_count: {faculty_course_count}")

        # # Calculate penalties
        # # Overloading Faculty
        # if self.teacher_type == 'FT' and total_hours > self.max_hours:
        #     penalty += 10

        # # Part-time Faculty
        # if self.teacher_type == 'PT' and total_hours > 12:
        #     penalty += 7

        # # Inequitable Course Distribution
        # print(f"faculty_course_count: {faculty_course_count}")
        # # faculty_course_count: {'FT6': 2, 'PT1': 3, 'PT4': 2, 'FT2': 1, 'FT3': 1, 'FT1': 1}

        # # Check if distribution is inequitable
        # if len(faculty_course_count) > 1:
        #     max_courses = max(faculty_course_count.values())
        #     min_courses = min(faculty_course_count.values())
        #     error_percentage = (max_courses - min_courses) / max_courses

        #     if error_percentage > 0.1:
        #         penalty += int(error_percentage * 10)  # Adjust the penalty based on the error percentage

        # # Consulting Hours Overlap
        # for i in range(len(self.assignments)):
        #     for j in range(i + 1, len(self.assignments)):
        #         if self.assignments[i]['faculty'] == self.assignments[j]['faculty'] and \
        #                 self.assignments[i]['timeslot'] == self.assignments[j]['timeslot']:
        #             penalty += 5
        #             break

        # # Faculty Preferences Ignored
        # for a in self.assignments:
        #     if isinstance(a, dict) and 'day' in a and a['day'] in self.unavailable_days:
        #         penalty += 4

        # classroom_usage = {}
        # for a in self.assignments:
        #     if isinstance(a, dict):
        #         classroom_usage[a['classroom']] = classroom_usage.get(a['classroom'], 0) + self.calculate_duration(a['start_time'], a['end_time'])

        # for classroom, usage in classroom_usage.items():
        #     if usage < 3:
        #         penalty += 3
        #     elif usage > 5:
        #         penalty += 3

        # # Student Schedule Clashes
        # for i in range(len(self.assignments)):
        #     for j in range(i + 1, len(self.assignments)):
        #         if self.assignments[i]['timeslot'] == self.assignments[j]['timeslot'] and \
        #                 self.assignments[i]['classroom'] == self.assignments[j]['classroom']:
        #             penalty += 2
        #             break

        # # Compute fitness score and subtract penalties
        # fitness_score = 1 / (1 + penalty)

        # Integrate TDA fitness evaluation
        point_cloud = self.to_point_cloud()

        # print(f"point_cloud: {point_cloud}")

        persistence_diagrams = compute_persistence_diagrams(point_cloud)

        # print(f"persistence_diagrams: {persistence_diagrams}")

        final_fitness = compute_tda_fitness(persistence_diagrams)

        print(f"4 final_fitness: {final_fitness}")

        return 1


        
# NOTE: Slows down inside compute_persistence_diagrams() method
# TDA-related functions
def compute_persistence_diagrams(point_cloud):
    print("1 Inside compute_persistence_diagrams()")

    # Creating Alpha Complex and Simplex Tree
    alpha_complex = gd.AlphaComplex(points=point_cloud)
    simplex_tree = alpha_complex.create_simplex_tree(max_alpha_square=42)  # Adjust max_alpha_square if necessary

    # Compute persistence and filter out infinite values in a more efficient way
    persistence = simplex_tree.persistence(min_persistence=0.0)  # Ignoring features with 0 persistence
    persistence_diagrams = [p for p in persistence if p[1] != float('inf')]

    print(f"2 persistence_diagrams: {persistence_diagrams}")

    return persistence_diagrams

def compute_tda_fitness(persistence_diagrams):
    """
    Compute the TDA fitness value from the persistence diagrams.
    """
    fitness_value = sum(p[1][1] - p[1][0] for p in persistence_diagrams if p[1][1] != float('inf'))
    print(f"3 compute_tda_fitness_value: {fitness_value}")
    return fitness_value

In [688]:
# Initialize population
def initialize_population(size, df, teacher_type, max_hours, unavailable_days, location_to_index_map):
    population = []
    for _ in range(size):
        assignments = df.sample(n=10).to_dict('records') # Adjust the number of assignments as necessary
        population.append(Schedule(assignments, teacher_type, '1', max_hours, unavailable_days, location_to_index_map))

    print(f"== population len: {len(population)} | size: {size} ==")   
    return population

In [689]:
def select_parents(population, num_elites):
  # Sort the population by fitness in descending order
  sorted_population = sorted(population, key=lambda x: x.fitness(), reverse=True)

  # Select the top num_elites individuals as the elites
  elites = elitism(sorted_population, num_elites)

  # Remove the elites from the sorted population
  sorted_population = [individual for individual in sorted_population if individual not in elites]

  # Calculate selection probabilities based on ranks
  total_ranks = sum(range(1, len(sorted_population) + 1)) # Sum of ranks
  selection_probabilities = [rank / total_ranks for rank in range(len(sorted_population), 0, -1)]

  # Select parents
  parents = []
  for _ in range(len(sorted_population)):
      chosen_parent = random.choices(sorted_population + elites, weights=selection_probabilities, k=1)[0]
      parents.append(chosen_parent)

  return parents


In [690]:
# crossover
def enhanced_crossover(parent1, parent2, teacher_type, max_hours, unavailable_days):
        """
        Performs enhanced crossover between two parent schedules. Crossover is performed by selecting two random crossover.

        Parameters:
        - parent1 (Schedule): The first parent schedule.
        - parent2 (Schedule): The second parent schedule.
        - teacher_type (str): The type of teacher (FT or PT).
        - max_hours (int): The maximum number of hours for full-time teachers.
        - unavailable_days (list): The list of unavailable days for faculty.

        Returns:
        - child (Schedule): The child schedule generated from crossover.
        """

        # Select two random crossover points
        crossover_point1 = random.randint(1, len(parent1.assignments) - 2)
        crossover_point2 = random.randint(crossover_point1 + 1, len(parent1.assignments) - 1)

        new_assignments = parent1.assignments[:crossover_point1] + parent2.assignments[crossover_point1:crossover_point2] + parent1.assignments[crossover_point2:]
        child = Schedule(new_assignments, teacher_type, ga_type, max_hours, unavailable_days)

        return child


In [691]:
# Mutation
def mutate(schedule):
    mutation_rate = 0.1
    if random.random() < mutation_rate:
        mutated_assignment = df_sampled.sample(1).to_dict('records')[0]
        schedule.assignments[random.randint(0, len(schedule.assignments) - 1)] = mutated_assignment
        # print(f"Mutation occurred with new assignment: {mutated_assignment}")
    return schedule

In [692]:
# Elitism
def elitism(population, top_k=1):
    return sorted(population, key=lambda s: s.fitness(), reverse=True)[:top_k]

# PROPOSED GA

In [693]:
def logistic_map(r, x):
    """
    Calculate the logistic map value based on the given parameters.

    Parameters:
    - r (float): The growth rate parameter. Expected to be in the range [0, 4].
    - x (float): The current value. Expected to be in the range [0, 1].

    Returns:
    - float: The logistic map value. Expected to be in the range [0, 1].
    """
    result = r * x * (1 - x)
    print("[LM] Logistic Map Result:", result)
    return result

def chaotic_initial_population(size, df, teacher_type, max_hours, unavailable_days, location_to_index_map):
    print("[CIP] Initializing Chaotic Initial Population")

    r = 3.99  # Chaotic regime for logistic map
    x = 0.5  # Arbitrary start value in (0, 1)
    population = []

    # Sample a sufficient number of rows from df using numpy
    x = logistic_map(r, x)
    raw_sample_size = int(x * len(df))  # Calculate raw sample size
    print("[CIP] Raw Sample Size:", raw_sample_size)
    
    sample_size = min(raw_sample_size, len(df))  # Ensure sample size does not exceed the length of df
    print("[CIP] Final Sample Size:", sample_size)

    sampled_indices = np.random.choice(df.index, size=sample_size, replace=False)
    sampled_df = df.loc[sampled_indices].to_dict('records')  # Convert to a list of dictionaries

    # Create Schedule instances
    for i in range(size):
        assignments = sampled_df[i * size:(i + 1) * size]  # Get the assignments for the current individual
        schedule = Schedule(assignments, teacher_type, '2', max_hours, unavailable_days, location_to_index_map)  # Pass '2' for proposed GA
        population.append(schedule)
        print("[CIP] Schedule created for individual", i + 1)

    print("[CIP] Finishing Chaotic Initial Population")
    return population


In [694]:

# NOTE: Slows down inside compute_persistence_diagrams() method
# Implement TDA analysis using Gudhi
def compute_persistence_diagrams(point_cloud):
    # print(f"0 Point Cloud: {point_cloud}")

    # print("1 Inside compute_persistence_diagrams()")

    # print("2 Invoking gd.AlphaComplex()")
    alpha_complex = gd.AlphaComplex(points=point_cloud)
    """
        Alpha complex is a simplicial complex associated to a finite set of points in a metric space. 
        It is constructed by attaching tetrahedra to the points in such a way that the tetrahedra associated to nearby points fit together.
    """
    # print(f"3 alpha_complex: {alpha_complex}")

    # print("4 Invoking alpha_complex.create_simplex_tree()")
    simplex_tree = alpha_complex.create_simplex_tree()
    # print(f"5 simplex_tree: {simplex_tree}")

    # print("6 Invoking simplex_tree.persistence()")
    persistence = simplex_tree.persistence()
    # print(f"7 persistence: {persistence}")
    
    persistence_diagrams = []

    for p in persistence:
        if p[0] == 0:
            continue
        persistence_diagrams.append(p)

    # print(f"8 persistence_diagrams: {persistence_diagrams}")

    return persistence_diagrams


In [695]:
def check_termination_condition(population, generation, max_generations, target_fitness, convergence_threshold):
    """
    Check if the termination condition for the genetic algorithm is met.

    :param population: The current population of schedules.
    :param generation: The current generation number.
    :param max_generations: The maximum number of generations to run.
    :param target_fitness: The target fitness score to achieve.
    :param convergence_threshold: The threshold for population convergence.

    :return: True if the termination condition is met, False otherwise.
    """

    # Print params for checking
    # print(f"(Inside Check Termination Condition) Generation: {generation}, max_generations: {max_generations}, target_fitness: {target_fitness}, convergence_threshold: {convergence_threshold}")

    # Check if the maximum number of generations has been reached
    if generation >= max_generations:
        # print(f"1 Terminated at (generation >= max_generations) | {generation} >= {max_generations}")

        return True

    # Check if any individual in the population has reached the target fitness
    if any(schedule.fitness() >= target_fitness for schedule in population):
        print(f"2 Terminated at (any(schedule.fitness() >= target_fitness for schedule in population)) | {any(schedule.fitness() >= target_fitness for schedule in population)}")

        return True

    # print(f"Populations: ${population}")

    # # Check for population convergence
    # # fitness_values = [schedule.fitness() for schedule in population]

    # fitness_values = []

    # # Loop through population and get the fitness values instead of one line
    # for schedule in population:
    #     print(f"(Inside a loop in population) Schedule: {schedule}")
    #     print(f"(Inside a loop in population) Fitness: {schedule.fitness()}")
    #     print(f"(Inside a loop in population) Fitness Values: {fitness_values}")

    #     fitness_values.append(schedule.fitness())

    # if max(fitness_values) - min(fitness_values) < convergence_threshold:

    #     print(f"Fitness Values: ${fitness_values}")

    #     print(f"Max Fitness Values: {max(fitness_values)}")
    #     print(f"Min Fitness Values: {min(fitness_values)}")

    #     print(f"Convergence Threshold: {convergence_threshold}")

    #     print(f"3 Terminated at (max(fitness_values) - min(fitness_values) < convergence_threshold) | {max(fitness_values) - min(fitness_values) < convergence_threshold}")

    #     return True
    
    # print("4 Continuing...")

    # If none of the above conditions are met, continue the algorithm
    return False


In [696]:
def check_convergence_with_enhanced_criteria(population, fitness_threshold, diversity_threshold, improvement_threshold, num_generations_for_improvement_check):
    print(f"Inside check_convergence_with_enhanced_criteria | Population: {population} | Fitness Threshold: {fitness_threshold} | Diversity Threshhold: {diversity_threshold} | Improvement Threshold: {improvement_threshold} | Num Generations For Improvement Check: {num_generations_for_improvement_check}")
    
    # Check if any individual exceeds the fitness threshold
    # if any(individual.fitness() >= fitness_threshold for individual in population):
    #     print(f"1 Terminated at (any(individual.fitness() >= fitness_threshold for individual in population)) | {any(individual.fitness() >= fitness_threshold for individual in population)}")
    #     return True

    # Calculate diversity (could be based on genetic diversity or variance in fitness)
    fitness_values = [individual.fitness() for individual in population]
    print(f"Fitness Values: {fitness_values}")

    diversity = calculate_diversity(fitness_values)  # Implement this function
    print(f"Diversity: {diversity}")

    # Check if diversity is too low
    if diversity < diversity_threshold:
        print(f"2 Terminated at (diversity < diversity_threshold) | {diversity < diversity_threshold}")
        return True

    # Check improvement rate (implement a way to track fitness over generations)
    if len(fitness_values) > num_generations_for_improvement_check:
        print(f"3 Terminated at (len(fitness_values) > num_generations_for_improvement_check) | {len(fitness_values) > num_generations_for_improvement_check}")

        recent_fitness_values = fitness_values[-num_generations_for_improvement_check:]
        print(f"Recent Fitness Values: {recent_fitness_values}")

        improvement = max(recent_fitness_values) - min(recent_fitness_values)
        print(f"Improvement: {improvement}")

        if improvement < improvement_threshold:
            print(f"4 Terminated at (improvement < improvement_threshold) | {improvement < improvement_threshold}")
            return True
    
    print("5 Continuing...")
    return False

def calculate_diversity(fitness_values):
    # Implement a method to calculate diversity.
    # For example, using standard deviation as a measure of diversity
    return np.std(fitness_values)

In [697]:
def de_crossover_and_mutation(parent1, parent2, population, F=0.8, CR=0.5):
    # Inside de_crossover_and_mutation
    print(f"Inside de_crossover_and_mutation | parent1: {parent1} | parent2: {parent2} | population: {population} | F: {F} | CR: {CR}")

    # Select two random individuals from the population for differential mutation
    individual1, individual2 = random.sample(population, 2)
    # print(f"individual1: {individual1} | individual2: {individual2}")

    # Ensure lengths are the same
    min_length = min(len(parent1.assignments), len(individual1.assignments), len(individual2.assignments))

    # DE Trial Vector Creation
    trial_vector = {}

    print('starting loop for key in parent1.assignments')
    for i in range(min_length):
        print(f'parent1.assignments: {parent1.assignments}')
        print(f'key: {parent1.assignments[i]}')

        base_value = parent1.assignments[i]

        print(f'individual1.assignments: {individual1.assignments}')
        print(f'individual2.assignments: {individual2.assignments}')
        print(f'base_value: {base_value}')
        print(f'F: {F}')

        # Perform element-wise subtraction for each key in the dictionaries
        assignment_difference = {}
        # Inside the loop for key in base_value
        for key in base_value:
            # Check if the value is numeric before conversion
            if key.isnumeric():
                assignment_difference[key] = float(individual1.assignments[i][key]) - float(individual2.assignments[i][key])
            else:
                assignment_difference[key] = 0  # Set a default value for non-numeric keys

        trial_vector[i] = {}
        for key in base_value:
            trial_vector[i][key] = str(base_value[key]) + str(F * assignment_difference[key])

    print(f"trial_vector: {trial_vector}")

    # DE Crossover Operation
    child_assignments = {}
    for i in range(min_length):
        if random.random() < CR:
            child_assignments[i] = trial_vector[i]
        else:
            child_assignments[i] = parent1.assignments[i]
    print(f"child_assignments: {child_assignments}")

    # Create child with new assignments
    # child = Schedule(child_assignments, parent1.teacher_type, parent1.ga_type, parent1.max_hours, parent1.unavailable_days)
    child = Schedule(child_assignments, parent1.teacher_type, ga_type, parent1.max_hours, parent1.unavailable_days)
    # print(f"child: {child}")

    # Mutation (optional, depending on your design)
    child = mutate(child)  # Assuming 'mutate' is a predefined function
    # print(f"child: {child}")

    return child


In [698]:
def dynamic_parameter_adjustment_with_tda(schedule, population, feature_persistence_threshold, location_to_index_map):
    print("[DPAT] 1 Inside dynamic_parameter_adjustment_with_tda()")

    # Convert the population to a point cloud for TDA
    point_cloud = []
    for idx, individual in enumerate(population):
        individual_cloud = individual.to_point_cloud()
        point_cloud.extend(individual_cloud)  # Flatten the structure by extending the main point cloud
    print(f"[DPAT] 2 Point Cloud: {point_cloud}")

    # Perform TDA on the point cloud
    persistence_diagrams = compute_persistence_diagrams(point_cloud)
    print(f"[DPAT] 3 Persistence Diagrams: {persistence_diagrams}")

    # Analyze the persistence diagrams
    analysis_result = analyze_persistence_diagrams(persistence_diagrams, feature_persistence_threshold)
    print(f"[DPAT] 4 Analysis Result: {analysis_result}")

    # Adjust parameters based on TDA analysis
    if analysis_result['indicates_convergence']:
        print("[DPAT] 5 Analysis Result indicates convergence")

        # Increase mutation rate to maintain diversity
        # schedule.mutation_rate = min(schedule.mutation_rate + 0.05, 1.0)

        # print(f"[DPAT] 6 Schedule Mutation Rate: {schedule.mutation_rate}")

    elif analysis_result['indicates_diversity']:
        print("[DPAT] 7 Analysis Result indicates diversity")

        # Decrease mutation rate
        # schedule.mutation_rate = max(schedule.mutation_rate - 0.05, 0.01)

        # print(f"[DPAT] 8 Schedule Mutation Rate: {schedule.mutation_rate}")

    return schedule


def analyze_persistence_diagrams(persistence_diagrams, feature_persistence_threshold):
    # Define thresholds for convergence and diversity
    CONVERGENCE_THRESHOLD = 10  # Example value indicating a high level of population complexity
    DIVERSITY_THRESHOLD = 5     # Example value indicating a low level of population complexity

    # Count significant features in persistence diagrams
    num_significant_features = len([d for d in persistence_diagrams if d[1][1] - d[1][0] > feature_persistence_threshold])

    # Determine population structure based on significant features
    analysis_result = {
        'indicates_convergence': num_significant_features > CONVERGENCE_THRESHOLD,
        'indicates_diversity': num_significant_features < DIVERSITY_THRESHOLD,
    }

    return analysis_result


In [699]:
def select_best_solution(population):
    # Select the individual with the highest fitness score in the population
    best_individual = max(population, key=lambda individual: individual.fitness())
    return best_individual


In [700]:
def plot_fitness_scores(fitness_scores, title):
    plt.plot(fitness_scores)
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness Score')
    plt.title(title)
    plt.show()

# GENERIC GA FUNCTION

In [701]:
target_fitness = 95  # This means we are aiming for a solution with at least a fitness of 95
convergence_threshold = 0.5  # This means we consider the algorithm to have converged if the improvement is less than 0.5

def generic_GA(population_size, max_generations, df, teacher_type, max_hours=None, unavailable_days=None):
    # Print parameters for tracking
    print(f"Population size: {population_size}, max generations: {max_generations}, teacher type: {teacher_type}, max hours: {max_hours}, unavailable days: {unavailable_days}")

    # Define the mapping from classroom locations to indices
    location_to_index_map = {classroom: index for index, classroom in enumerate(df['classroom'].unique(), start=1)}

    # Initialize population
    population = initialize_population(population_size, df, teacher_type, max_hours, unavailable_days, location_to_index_map)

    fitness_scores = []  # List to store the best fitness score in each generation

    # # Main GA loop
    # for generation in range(max_generations):
    #     # Append the best fitness score in the current generation, inside population.fitnes()
    #     generation_best_fitness_without_max = [individual.fitness() for individual in population] # May 1.0 sa loob ng list na 'to, have to fix fitness. Masyadong unrealistic
    #     print(f"Generation Best Fitness Without Max: {generation_best_fitness_without_max} | generation_best_fitness_without_max len {len(generation_best_fitness_without_max)}")

    #     generation_best_fitness = max(individual.fitness() for individual in population)
    #     print(f"Generation Best Fitness: {generation_best_fitness}")

    #     fitness_scores.append(generation_best_fitness)

    #     print(f"Fitness Scores: {fitness_scores}")

    #     # Check termination condition
    #     if check_termination_condition(population, generation, max_generations, target_fitness, convergence_threshold):
    #         print(f"Termination condition met at generation {generation}.")
    #         break
        
    #     # Select parents -- under select_parents() function
    #     print(f"Selecting parents for generation {generation}...")

    #     # Sort the population by fitness in descending order and assign ranks
    #     sorted_population = sorted(population, key=lambda x: x.fitness(), reverse=True)
        
    #     # Calculate selection probabilities based on ranks
    #     total_ranks = sum(range(1, len(population) + 1))  # Sum of ranks
    #     selection_probabilities = [rank / total_ranks for rank in range(len(population), 0, -1)]
    #     print(f"Selection Probabilities: {selection_probabilities}")

    #     # Get sorted population in an array of their fitness values
    #     sorted_population_fitness = [individual.fitness() for individual in sorted_population]
    #     print(f"Sorted Population Fitness: {sorted_population_fitness}")
        

    #     # Select parents ---
    #     parents = []
    #     for _ in range(len(population)):
    #         chosen_parent = random.choices(sorted_population, weights=selection_probabilities, k=1)[0]
    #         parents.append(chosen_parent)
    #         # print(f"Parent selected with fitness: {chosen_parent.fitness()}")

    #     # Loop through parents and print their fitness
    #     for parent in parents:
    #         print(f"Parent Fitness: {parent.fitness()}")

    #     # ---

    #     # Generate offspring through crossover, handling odd number of parents ---
    #     children = []
        
    #     for i in range(0, len(parents), 2):
    #         # Check if the next parent exists
    #         if i + 1 < len(parents): # If the next parent exists
    #             # Print 2 parents and their fitness
    #             print(f"Parent 1: {parents[i]} | Parent 1 Fitness: {parents[i].fitness()}")
    #             print(f"Parent 2: {parents[i+1]} | Parent 2 Fitness: {parents[i+1].fitness()}")

    #             child = enhanced_crossover(parents[i], parents[i+1], teacher_type, max_hours, unavailable_days)

    #             # Print the child and its fitness
    #             print(f"Child: {child} | Child Fitness: {child.fitness()}")

    #             children.append(child)
    #         else: # Handle the case where the number of parents is odd
    #             # Example: Pair the last parent with the first parent
    #             child = enhanced_crossover(parents[i], parents[0], teacher_type, max_hours, unavailable_days)
    #             children.append(child)

    #     # ---

    #     # Mutate the offspring
    #     for child in children:
    #         mutate(child)

    #     # Apply elitism to form the next generation, this preserves the best individuals
    #     elite = elitism(population)
    #     population = children + elite

    #     # Loop through population and print their fitness
    #     for individual in population:
    #         print(f"1 Individual Fitness: {individual.fitness()}")

    #     # Logging for tracking
    #     print(f"Generation {generation}: Best fitness score: {generation_best_fitness}")



    # Find the best solution from the final population
    best_solution = max(population, key=lambda ind: ind.fitness())

    # Plot the evolution of fitness scores
    plt.plot(fitness_scores)
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness Score')
    plt.title('Generic Genetic Algorithm Progress')
    plt.show()

    return best_solution


In [702]:
def proposed_GA(population_size, max_generations, df, teacher_type, max_hours=None, unavailable_days=None, feature_persistence_threshold=0.1):

    # Print all parameters
    print(f"[PGA] Population size: {population_size}, max generations: {max_generations}, teacher type: {teacher_type}, max hours: {max_hours}, unavailable days: {unavailable_days}, feature persistence threshold: {feature_persistence_threshold}")

    # Define the mapping from classroom locations to indices
    location_to_index_map = {classroom: index for index, classroom in enumerate(df['classroom'].unique(), start=1)}

    # population = chaotic_initial_population(population_size, df, teacher_type, max_hours, unavailable_days, location_to_index_map)

    # NOTE: Temporary fix for chaotic_initial_population() function
    population = initialize_population(population_size, df, teacher_type, max_hours, unavailable_days, location_to_index_map)
    
    fitness_scores = []
    
    # # Main GA loop
    # for generation in range(max_generations):
    #     # Append the best fitness score in the current generation, inside population.fitnes()
    #     generation_best_fitness_without_max = [individual.fitness() for individual in population] # May 1.0 sa loob ng list na 'to, have to fix fitness. Masyadong unrealistic
    #     print(f"Generation Best Fitness Without Max: {generation_best_fitness_without_max} | generation_best_fitness_without_max len {len(generation_best_fitness_without_max)}")

    #     generation_best_fitness = max(individual.fitness() for individual in population)
    #     print(f"Generation Best Fitness: {generation_best_fitness}")

    #     fitness_scores.append(generation_best_fitness)

    #     print(f"Fitness Scores: {fitness_scores}")

    #     # Check termination condition
    #     if check_termination_condition(population, generation, max_generations, target_fitness, convergence_threshold):
    #         print("[PGA] Termination condition met. Exiting.")
    #         break

    #     # Sort the population by fitness in descending order and assign ranks
    #     sorted_population = sorted(population, key=lambda x: x.fitness(), reverse=True)
        
    #     # Calculate selection probabilities based on ranks
    #     total_ranks = sum(range(1, len(population) + 1))  # Sum of ranks
    #     selection_probabilities = [rank / total_ranks for rank in range(len(population), 0, -1)]
    #     print(f"Selection Probabilities: {selection_probabilities}")

    #     # Get sorted population in an array of their fitness values
    #     sorted_population_fitness = [individual.fitness() for individual in sorted_population]
    #     print(f"Sorted Population Fitness: {sorted_population_fitness}")

    #     # Select parents ---

    #     parents = []
        
    #     for _ in range(len(population)):
    #         chosen_parent = random.choices(sorted_population, weights=selection_probabilities, k=1)[0]
    #         parents.append(chosen_parent)
    #         print(f"Parent selected with fitness: {chosen_parent.fitness()}")

    #     # Loop through parents and print their fitness
    #     for parent in parents:
    #         print(f"Parent Fitness: {parent.fitness()}")

    #     # ---

    #     # # Initialize thresholds for dynamic parameter adjustment
    #     # fitness_threshold = 1
    #     # diversity_threshold = 1
    #     # improvement_threshold = 1
    #     # num_generations_for_improvement_check = 100

    #     # # Generate offspring through crossover
    #     # children = []
    #     # if len(parents) >= 2:
    #     #     children = [de_crossover_and_mutation(parents[i], parents[i+1], population) for i in range(0, len(parents)-1, 2)]
    #     # else:
    #     #     # If there are fewer than two parents, generate children using a different method
    #     #     # For example, you can generate children by copying the parents, or by applying mutation to the parents

    #     #     # Copy the parents
    #     #     children = parents.copy()

    #     #     # Apply mutation to the children
    #     #     children = [mutate(child) for child in children]

    #     # # Apply DE-Inspired crossover and mutation or dynamic parameter adjustment based on convergence
    #     # # print("[PGA] Applying crossover and mutation...")
    #     # if not check_convergence_with_enhanced_criteria(population, fitness_threshold, diversity_threshold, improvement_threshold, num_generations_for_improvement_check):
    #     #     # Add the children generated by the dynamic_parameter_adjustment_with_tda function to the existing children
    #     #     children += [dynamic_parameter_adjustment_with_tda(child, population, feature_persistence_threshold, location_to_index_map) for child in children]

    #     # Generate offspring through crossover, handling odd number of parents ---
    #     # children = []
        
    #     # for i in range(0, len(parents), 2):
    #     #     # Check if the next parent exists
    #     #     if i + 1 < len(parents): # If the next parent exists
    #     #         # Print 2 parents and their fitness
    #     #         print(f"Parent 1: {parents[i]} | Parent 1 Fitness: {parents[i].fitness()}")
    #     #         print(f"Parent 2: {parents[i+1]} | Parent 2 Fitness: {parents[i+1].fitness()}")

    #     #         child = enhanced_crossover(parents[i], parents[i+1], teacher_type, max_hours, unavailable_days)

    #     #         # Print the child and its fitness
    #     #         print(f"Child: {child} | Child Fitness: {child.fitness()}")

    #     #         children.append(child)
    #     #     else: # Handle the case where the number of parents is odd
    #     #         # Example: Pair the last parent with the first parent
    #     #         child = enhanced_crossover(parents[i], parents[0], teacher_type, max_hours, unavailable_days)
    #     #         children.append(child)

    #     # # ---

    #     # elite = elitism(population)

    #     # population = children + elite

    for generation in range(max_generations):
        # Append the best fitness score in the current generation, inside population.fitnes()
        generation_best_fitness_without_max = [individual.fitness() for individual in population] # May 1.0 sa loob ng list na 'to, have to fix fitness. Masyadong unrealistic
        print(f"Generation Best Fitness Without Max: {generation_best_fitness_without_max} | generation_best_fitness_without_max len {len(generation_best_fitness_without_max)}")

        generation_best_fitness = max(individual.fitness() for individual in population)
        print(f"Generation Best Fitness: {generation_best_fitness}")

        fitness_scores.append(generation_best_fitness)

        print(f"Fitness Scores: {fitness_scores}")

        # Check termination condition
        if check_termination_condition(population, generation, max_generations, target_fitness, convergence_threshold):
            print(f"Termination condition met at generation {generation}.")
            break
        
        # Select parents -- under select_parents() function
        print(f"Selecting parents for generation {generation}...")

        # Sort the population by fitness in descending order and assign ranks
        sorted_population = sorted(population, key=lambda x: x.fitness(), reverse=True)
        
        # Calculate selection probabilities based on ranks
        total_ranks = sum(range(1, len(population) + 1))  # Sum of ranks
        selection_probabilities = [rank / total_ranks for rank in range(len(population), 0, -1)]
        print(f"Selection Probabilities: {selection_probabilities}")

        # Get sorted population in an array of their fitness values
        sorted_population_fitness = [individual.fitness() for individual in sorted_population]
        print(f"Sorted Population Fitness: {sorted_population_fitness}")
        

        # Select parents ---

        parents = []
        for _ in range(len(population)):
            chosen_parent = random.choices(sorted_population, weights=selection_probabilities, k=1)[0]
            parents.append(chosen_parent)
            print(f"Parent selected with fitness: {chosen_parent.fitness()}")

        # Loop through parents assignment and print their fitness
        for parent in parents:
            print(f"Parent Fitness: {parent.fitness()} | Parent Assignments: {parent.assignments}")

        # ---

        # Generate offspring through crossover, handling odd number of parents ---
        children = []
        
        # Create an offspring for each parent, crossover the assignment
        for i in range(0, len(parents), 2):
            # Check if the next parent exists
            if i + 1 < len(parents):

                crossover_point1 = random.randint(1, len(parents[i].assignments) - 2)
                crossover_point2 = random.randint(crossover_point1 + 1, len(parents[i].assignments) - 1)

                new_assignments = parents[i].assignments[:crossover_point1] + parents[i+1].assignments[crossover_point1:crossover_point2] + parents[i].assignments[crossover_point2:]

                child = Schedule(new_assignments, teacher_type, ga_type, max_hours, unavailable_days, location_to_index_map)

                children.append(child)
            else: # Handle the case where the number of parents is odd
                # Example: Pair the last parent with the first parent
                crossover_point1 = random.randint(1, len(parents[i].assignments) - 2)
                crossover_point2 = random.randint(crossover_point1 + 1, len(parents[i].assignments) - 1)

                new_assignments = parents[i].assignments[:crossover_point1] + parents[0].assignments[crossover_point1:crossover_point2] + parents[i].assignments[crossover_point2:]

                child = Schedule(new_assignments, teacher_type, ga_type, max_hours, unavailable_days, location_to_index_map)

                children.append(child)

        # ---        

        # Apply elitism to form the next generation, this preserves the best individuals
        elite = elitism(population)
        population = children + elite

        # Loop through population and print their fitness
        for individual in population:
            print(f"1 Individual Fitness: {individual.fitness()}")

        # Logging for tracking
        print(f"Generation {generation}: Best fitness score: {generation_best_fitness}")


    # Find the best solution from the final population
    best_solution = select_best_solution(population)
    print(f"[PGA] best_solution: {best_solution}")

    # Plot the evolution of fitness scores
    plt.plot(fitness_scores)
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness Score')
    plt.title('Proposed Genetic Algorithm Progress')
    plt.show()

    return best_solution


In [703]:
teacher_type = input("Enter teacher type (FT for Full-Time, PT for Part-Time): ").strip()
# teacher_type = "FT"

max_hours = None
unavailable_days = []

if teacher_type == 'FT':
    max_hours = float(input("Enter maximum hours you can teach per week: "))
    # max_hours = 30
elif teacher_type == 'PT':
    unavailable_days = input("Enter unavailable days (comma-separated, e.g., Monday,Wednesday): ").split(',')
    # unavailable_days =  "Monday,Wednesday".split(',')


In [704]:
def run_genetic_algorithm(ga_type, population_size, max_generations, df, teacher_type, max_hours=None, unavailable_days=None, feature_persistence_threshold=0.1):
    # Convert ga_type to string to ensure consistency in comparison
    ga_type = str(ga_type)
    
    if ga_type == '1':  # If generic GA is selected
        return generic_GA(population_size, max_generations, df, teacher_type, max_hours, unavailable_days)
    elif ga_type == '2':  # If proposed GA is selected
        return proposed_GA(population_size, max_generations, df, teacher_type, max_hours, unavailable_days, feature_persistence_threshold)
    else:
        raise ValueError(f"Encountered invalid GA type: {ga_type}")

# Define the parameters for the genetic algorithm
population_size = 100  # Set the population size as per your requirement
max_generations = 100   # Set the maximum number of generations

# Run the chosen genetic algorithm to find the best schedule
best_schedule_found = run_genetic_algorithm(ga_type, population_size, max_generations, df_sampled, teacher_type, max_hours, unavailable_days)

# Check if a schedule was found and print details
if best_schedule_found:
    print("Best schedule fitness:", best_schedule_found.fitness())
    print("Best Schedule Details:\n", best_schedule_found)
else:
    print("No suitable schedule was found.")


[PGA] Population size: 100, max generations: 100, teacher type: FT, max hours: 30.0, unavailable days: [], feature persistence threshold: 0.1
Test
Processed assignment: {'course': 'CSC 0311', 'faculty': 'PT4', 'timeslot': 'Tuesday 5:00 - 6:00 PM', 'classroom': 'GV 301', 'day': 'Tuesday', 'time': '5:00 - 6:00 PM', 'start_time': '5:00 ', 'end_time': ' 6:00 PM'}, Time Float: 5.0, Classroom Index: None
Test
Processed assignment: {'course': 'CSC 0311', 'faculty': 'PT1', 'timeslot': 'Tuesday 9:00 - 10:00 AM', 'classroom': 'GV 305', 'day': 'Tuesday', 'time': '9:00 - 10:00 AM', 'start_time': '9:00 ', 'end_time': ' 10:00 AM'}, Time Float: 9.0, Classroom Index: None
Test
Processed assignment: {'course': 'CSC 0102', 'faculty': 'FT3', 'timeslot': 'Tuesday 10:00 - 11:00 AM', 'classroom': 'GV 306', 'day': 'Tuesday', 'time': '10:00 - 11:00 AM', 'start_time': '10:00 ', 'end_time': ' 11:00 AM'}, Time Float: 10.0, Classroom Index: None
Test
Processed assignment: {'course': 'CSC 0102', 'faculty': 'FT5', 

TypeError: must be real number, not NoneType